<p style="text-align: center;">МИНИСТЕРСТВО ОБРАЗОВАНИЯ И НАУКИ
РОССИЙСКОЙ ФЕДЕРАЦИИ

<p style="text-align: center;">Федеральное государственное автономное
образовательное учреждение высшего образования
«Самарский национальный исследовательский университет
имени академика С. П. Королева»
(Самарский университет)</p>
<br>
<br>
<br>

<p style="text-align: center;">Институт информатики и кибернетики
    
<p style="text-align: center;">Факультет информатики
    
<p style="text-align: center;">Кафедра программных систем
    
 <br><br><br>   

<p style="text-align: center;">ОТЧЁТ

<p style="text-align: center;">по лабораторной работе № 1
<p style="text-align: center;">«Введение в модель MapReduce»
<p style="text-align: center;">по курсу «Интеллектуальный анализ и большие данные»

<p style="text-align: center;">
<br><br><br><br><br><br><br><br>







<p style="text-align: right;">Выполнил: Лазарев М.Ю.
<p style="text-align: right;">гр. 6132-020402D
<p style="text-align: right;">
<br><br><br><br><br><br><br><br><br>








<p style="text-align: center;">Самара 2025


# Введение в модель MapReduce на Python


In [1]:
from typing import NamedTuple # requires python 3.6+
from typing import Iterator

In [2]:
def MAP(_, row:NamedTuple):
  if (row.gender == 'female'):
    yield (row.age, row)
    
def REDUCE(age:str, rows:Iterator[NamedTuple]):
  sum = 0
  count = 0
  for row in rows:
    sum += row.social_contacts
    count += 1
  if (count > 0):
    yield (age, sum/count)
  else:
    yield (age, 0)

Модель элемента данных

In [3]:
class User(NamedTuple):
  id: int
  age: str
  social_contacts: int
  gender: str

In [4]:
input_collection = [
    User(id=0, age=55, gender='male', social_contacts=20),
    User(id=1, age=25, gender='female', social_contacts=240),
    User(id=2, age=25, gender='female', social_contacts=500),
    User(id=3, age=33, gender='female', social_contacts=800)
]

Функция RECORDREADER моделирует чтение элементов с диска или по сети.

In [5]:
def RECORDREADER():
  return [(u.id, u) for u in input_collection]

In [6]:
RECORDREADER()

[(0, User(id=0, age=55, social_contacts=20, gender='male')),
 (1, User(id=1, age=25, social_contacts=240, gender='female')),
 (2, User(id=2, age=25, social_contacts=500, gender='female')),
 (3, User(id=3, age=33, social_contacts=800, gender='female'))]

In [7]:
def flatten(nested_iterable):
  for iterable in nested_iterable:
    for element in iterable:
      yield element

In [8]:
map_output = flatten(map(lambda x: MAP(*x), RECORDREADER()))
map_output = list(map_output) # materialize
map_output

[(25, User(id=1, age=25, social_contacts=240, gender='female')),
 (25, User(id=2, age=25, social_contacts=500, gender='female')),
 (33, User(id=3, age=33, social_contacts=800, gender='female'))]

In [9]:
list(map_output)

[(25, User(id=1, age=25, social_contacts=240, gender='female')),
 (25, User(id=2, age=25, social_contacts=500, gender='female')),
 (33, User(id=3, age=33, social_contacts=800, gender='female'))]

In [10]:
def groupbykey(iterable):
  t = {}
  for (k2, v2) in iterable:
    t[k2] = t.get(k2, []) + [v2]
  return t.items()

In [11]:
shuffle_output = groupbykey(map_output)
shuffle_output = list(shuffle_output)
shuffle_output

[(25,
  [User(id=1, age=25, social_contacts=240, gender='female'),
   User(id=2, age=25, social_contacts=500, gender='female')]),
 (33, [User(id=3, age=33, social_contacts=800, gender='female')])]

In [12]:
reduce_output = flatten(map(lambda x: REDUCE(*x), shuffle_output))
reduce_output = list(reduce_output)
reduce_output

[(25, 370.0), (33, 800.0)]

Все действия одним конвейером!

In [13]:
list(flatten(map(lambda x: REDUCE(*x), groupbykey(flatten(map(lambda x: MAP(*x), RECORDREADER()))))))

[(25, 370.0), (33, 800.0)]

# **MapReduce**
Выделим общую для всех пользователей часть системы в отдельную функцию высшего порядка. Это наиболее простая модель MapReduce, без учёта распределённого хранения данных. 

Пользователь для решения своей задачи реализует RECORDREADER, MAP, REDUCE.

In [14]:
def flatten(nested_iterable):
  for iterable in nested_iterable:
    for element in iterable:
      yield element

def groupbykey(iterable):
  t = {}
  for (k2, v2) in iterable:
    t[k2] = t.get(k2, []) + [v2]
  return t.items()

def MapReduce(RECORDREADER, MAP, REDUCE):
  return flatten(map(lambda x: REDUCE(*x), groupbykey(flatten(map(lambda x: MAP(*x), RECORDREADER())))))

## Спецификация MapReduce



```
f (k1, v1) -> (k2,v2)*
g (k2, v2*) -> (k3,v3)*
 
mapreduce ((k1,v1)*) -> (k3,v3)*
groupby ((k2,v2)*) -> (k2,v2*)*
flatten (e2**) -> e2*
 
mapreduce .map(f).flatten.groupby(k2).map(g).flatten
```




# Примеры

## SQL 

In [15]:
from typing import NamedTuple # requires python 3.6+
from typing import Iterator

class User(NamedTuple):
  id: int
  age: str
  social_contacts: int
  gender: str
    
input_collection = [
    User(id=0, age=55, gender='male', social_contacts=20),
    User(id=1, age=25, gender='female', social_contacts=240),
    User(id=2, age=25, gender='female', social_contacts=500),
    User(id=3, age=33, gender='female', social_contacts=800)
]

def MAP(_, row:NamedTuple):
  if (row.gender == 'female'):
    yield (row.age, row)
    
def REDUCE(age:str, rows:Iterator[NamedTuple]):
  sum = 0
  count = 0
  for row in rows:
    sum += row.social_contacts
    count += 1
  if (count > 0):
    yield (age, sum/count)
  else:
    yield (age, 0)
 
def RECORDREADER():
  return [(u.id, u) for u in input_collection]

output = MapReduce(RECORDREADER, MAP, REDUCE)
output = list(output)
output

[(25, 370.0), (33, 800.0)]

## Matrix-Vector multiplication 

In [16]:
from typing import Iterator
import numpy as np

mat = np.ones((5,4))
vec = np.random.rand(4) # in-memory vector in all map tasks

def MAP(coordinates:(int, int), value:int):
  i, j = coordinates
  yield (i, value*vec[j])
 
def REDUCE(i:int, products:Iterator[NamedTuple]):
  sum = 0
  for p in products:
    sum += p
  yield (i, sum)

def RECORDREADER():
  for i in range(mat.shape[0]):
    for j in range(mat.shape[1]):
      yield ((i, j), mat[i,j])
      
output = MapReduce(RECORDREADER, MAP, REDUCE)
output = list(output)
output

[(0, np.float64(1.9595716296562489)),
 (1, np.float64(1.9595716296562489)),
 (2, np.float64(1.9595716296562489)),
 (3, np.float64(1.9595716296562489)),
 (4, np.float64(1.9595716296562489))]

## Inverted index 

In [17]:
from typing import Iterator

d1 = "it is what it is"
d2 = "what is it"
d3 = "it is a banana"
documents = [d1, d2, d3]

def RECORDREADER():
  for (docid, document) in enumerate(documents):
    yield ("{}".format(docid), document)
      
def MAP(docId:str, body:str):
  for word in set(body.split(' ')):
    yield (word, docId)
 
def REDUCE(word:str, docIds:Iterator[str]):
  yield (word, sorted(docIds))

output = MapReduce(RECORDREADER, MAP, REDUCE)
output = list(output)
output

[('what', ['0', '1']),
 ('it', ['0', '1', '2']),
 ('is', ['0', '1', '2']),
 ('banana', ['2']),
 ('a', ['2'])]

## WordCount

In [18]:
from typing import Iterator

d1 = """
it is what it is
it is what it is
it is what it is"""
d2 = """
what is it
what is it"""
d3 = """
it is a banana"""
documents = [d1, d2, d3]

def RECORDREADER():
  for (docid, document) in enumerate(documents):
    for (lineid, line) in enumerate(document.split('\n')):
      yield ("{}:{}".format(docid,lineid), line)

def MAP(docId:str, line:str):
  for word in line.split(" "):  
    yield (word, 1)
 
def REDUCE(word:str, counts:Iterator[int]):
  sum = 0
  for c in counts:
    sum += c
  yield (word, sum)

output = MapReduce(RECORDREADER, MAP, REDUCE)
output = list(output)
output

[('', 3), ('it', 9), ('is', 9), ('what', 5), ('a', 1), ('banana', 1)]

In [19]:
list(RECORDREADER())

[('0:0', ''),
 ('0:1', 'it is what it is'),
 ('0:2', 'it is what it is'),
 ('0:3', 'it is what it is'),
 ('1:0', ''),
 ('1:1', 'what is it'),
 ('1:2', 'what is it'),
 ('2:0', ''),
 ('2:1', 'it is a banana')]

# MapReduce Distributed

Добавляется в модель фабрика RECORDREARER-ов --- INPUTFORMAT, функция распределения промежуточных результатов по партициям PARTITIONER, и функция COMBINER для частичной аггрегации промежуточных результатов до распределения по новым партициям.

In [20]:
def flatten(nested_iterable):
  for iterable in nested_iterable:
    for element in iterable:
      yield element

def groupbykey(iterable):
  t = {}
  for (k2, v2) in iterable:
    t[k2] = t.get(k2, []) + [v2]
  return t.items()
      
def groupbykey_distributed(map_partitions, PARTITIONER):
  global reducers
  partitions = [dict() for _ in range(reducers)]
  for map_partition in map_partitions:
    for (k2, v2) in map_partition:
      p = partitions[PARTITIONER(k2)]
      p[k2] = p.get(k2, []) + [v2]
  return [(partition_id, sorted(partition.items(), key=lambda x: x[0])) for (partition_id, partition) in enumerate(partitions)]
 
def PARTITIONER(obj):
  global reducers
  return hash(obj) % reducers
  
def MapReduceDistributed(INPUTFORMAT, MAP, REDUCE, PARTITIONER=PARTITIONER, COMBINER=None):
  map_partitions = map(lambda record_reader: flatten(map(lambda k1v1: MAP(*k1v1), record_reader)), INPUTFORMAT())
  if COMBINER != None:
    map_partitions = map(lambda map_partition: flatten(map(lambda k2v2: COMBINER(*k2v2), groupbykey(map_partition))), map_partitions)
  reduce_partitions = groupbykey_distributed(map_partitions, PARTITIONER) # shuffle
  reduce_outputs = map(lambda reduce_partition: (reduce_partition[0], flatten(map(lambda reduce_input_group: REDUCE(*reduce_input_group), reduce_partition[1]))), reduce_partitions)
  
  print("{} key-value pairs were sent over a network.".format(sum([len(vs) for (k,vs) in flatten([partition for (partition_id, partition) in reduce_partitions])])))
  return reduce_outputs

## Спецификация MapReduce Distributed


```
f (k1, v1) -> (k2,v2)*
g (k2, v2*) -> (k3,v3)*
 
e1 (k1, v1)
e2 (k2, v2)
partition1 (k2, v2)*
partition2 (k2, v2*)*
 
flatmap (e1->e2*, e1*) -> partition1*
groupby (partition1*) -> partition2*

mapreduce ((k1,v1)*) -> (k3,v3)*
mapreduce .flatmap(f).groupby(k2).flatmap(g)
```



## WordCount 

In [21]:
from typing import Iterator
import numpy as np

d1 = """
it is what it is
it is what it is
it is what it is"""
d2 = """
what is it
what is it"""
d3 = """
it is a banana"""
documents = [d1, d2, d3, d1, d2, d3]

maps = 3
reducers = 2

def INPUTFORMAT():
  global maps
  
  def RECORDREADER(split):
    for (docid, document) in enumerate(split):
      for (lineid, line) in enumerate(document.split('\n')):
        yield ("{}:{}".format(docid,lineid), line)
      
  split_size =  int(np.ceil(len(documents)/maps))
  for i in range(0, len(documents), split_size):
    yield RECORDREADER(documents[i:i+split_size])

def MAP(docId:str, line:str):
  for word in line.split(" "):  
    yield (word, 1)
 
def REDUCE(word:str, counts:Iterator[int]):
  sum = 0
  for c in counts:
    sum += c
  yield (word, sum)
  
# try to set COMBINER=REDUCE and look at the number of values sent over the network 
partitioned_output = MapReduceDistributed(INPUTFORMAT, MAP, REDUCE, COMBINER=None) 
partitioned_output = [(partition_id, list(partition)) for (partition_id, partition) in partitioned_output]
partitioned_output

56 key-value pairs were sent over a network.


[(0, [('', 6), ('a', 2), ('is', 18), ('it', 18), ('what', 10)]),
 (1, [('banana', 2)])]

In [22]:
partitioned_output = MapReduceDistributed(INPUTFORMAT, MAP, REDUCE, COMBINER=REDUCE) 
partitioned_output = [(partition_id, list(partition)) for (partition_id, partition) in partitioned_output]
partitioned_output

16 key-value pairs were sent over a network.


[(0, [('', 6), ('a', 2), ('is', 18), ('it', 18), ('what', 10)]),
 (1, [('banana', 2)])]

## TeraSort

In [23]:
import numpy as np

input_values = np.random.rand(30)
maps = 3
reducers = 2
min_value = 0.0
max_value = 1.0

def INPUTFORMAT():
  global maps
  
  def RECORDREADER(split):
    for value in split:
        yield (value, None)
      
  split_size =  int(np.ceil(len(input_values)/maps))
  for i in range(0, len(input_values), split_size):
    yield RECORDREADER(input_values[i:i+split_size])
    
def MAP(value:int, _):
  yield (value, None)
  
def PARTITIONER(key):
  global reducers
  global max_value
  global min_value
  bucket_size = (max_value-min_value)/reducers
  bucket_id = 0
  while((key>(bucket_id+1)*bucket_size) and ((bucket_id+1)*bucket_size<max_value)):
    bucket_id += 1
  return bucket_id

def REDUCE(value:int, _):
  yield (None,value)
  
partitioned_output = MapReduceDistributed(INPUTFORMAT, MAP, REDUCE, COMBINER=None, PARTITIONER=PARTITIONER)
partitioned_output = [(partition_id, list(partition)) for (partition_id, partition) in partitioned_output]
partitioned_output

30 key-value pairs were sent over a network.


[(0,
  [(None, np.float64(0.02368120145168562)),
   (None, np.float64(0.06041786880809963)),
   (None, np.float64(0.10419408512124773)),
   (None, np.float64(0.12421364810380697)),
   (None, np.float64(0.1308927544346007)),
   (None, np.float64(0.16104313263937808)),
   (None, np.float64(0.19571891579968925)),
   (None, np.float64(0.20253802148856215)),
   (None, np.float64(0.2298776958566402)),
   (None, np.float64(0.22996041204377493)),
   (None, np.float64(0.2362639459264605)),
   (None, np.float64(0.2623838291138748)),
   (None, np.float64(0.3224666015301897)),
   (None, np.float64(0.34227884434083145)),
   (None, np.float64(0.4307186839266227)),
   (None, np.float64(0.4572915757635009)),
   (None, np.float64(0.4805018232252508))]),
 (1,
  [(None, np.float64(0.5176216961731316)),
   (None, np.float64(0.6774263610600813)),
   (None, np.float64(0.7183119362745176)),
   (None, np.float64(0.738979597039996)),
   (None, np.float64(0.7495654265725202)),
   (None, np.float64(0.75436136848

In [24]:
partitioned_output = MapReduceDistributed(INPUTFORMAT, MAP, REDUCE, COMBINER=REDUCE, PARTITIONER=PARTITIONER)
partitioned_output = [(partition_id, list(partition)) for (partition_id, partition) in partitioned_output]
partitioned_output

TypeError: '>' not supported between instances of 'NoneType' and 'float'

In [25]:
import numpy as np

input_values = np.random.rand(30)
maps = 3
reducers = 2
min_value = 0.0
max_value = 1.0

def INPUTFORMAT():
    global maps
    
    def RECORDREADER(split):
        for value in split:
            yield (value, value)  # Теперь передаем сам value как key
    split_size = int(np.ceil(len(input_values) / maps))
    for i in range(0, len(input_values), split_size):
        yield RECORDREADER(input_values[i:i + split_size])

def MAP(value: float, key):
    yield (key, value)  # Ключ равен значению

def PARTITIONER(key):
    global reducers, max_value, min_value
    if key is None:
        raise ValueError("Ошибка: key в PARTITIONER == None!")  

    bucket_size = (max_value - min_value) / reducers
    bucket_id = int((key - min_value) / bucket_size)
    bucket_id = min(bucket_id, reducers - 1) 
    return bucket_id

def REDUCE(key, values):
    for value in values:
        yield (key, value) 

def groupbykey_distributed(map_partitions, PARTITIONER):
    partitions = [{} for _ in range(reducers)]
    for map_partition in map_partitions:
        for (k2, v2) in map_partition:
            p = partitions[PARTITIONER(k2)]
            p[k2] = p.get(k2, []) + [v2]
    return [(partition_id, sorted(partition.items(), key=lambda x: x[0])) for (partition_id, partition) in enumerate(partitions)]

def MapReduceDistributed(INPUTFORMAT, MAP, REDUCE, PARTITIONER, COMBINER=None):
    record_readers = list(INPUTFORMAT())
    map_outputs = map(lambda record_reader: flatten(map(lambda x: MAP(*x), record_reader)), record_readers)
    map_partitions = list(map(lambda x: list(x), map_outputs))  # Преобразуем в список для отладки

    if COMBINER is not None:
        map_partitions = map(lambda map_partition: flatten(map(lambda k2v2: COMBINER(*k2v2), groupbykey(map_partition))), map_partitions)

    reduce_partitions = groupbykey_distributed(map_partitions, PARTITIONER) 
    reduce_outputs = map(lambda reduce_partition: (reduce_partition[0], flatten(map(lambda reduce_input_group: REDUCE(*reduce_input_group), reduce_partition[1]))), reduce_partitions)

    return list(reduce_outputs)

partitioned_output = MapReduceDistributed(INPUTFORMAT, MAP, REDUCE, COMBINER=REDUCE, PARTITIONER=PARTITIONER)
partitioned_output = [(partition_id, list(partition)) for (partition_id, partition) in partitioned_output]
print("Partitioned Output:", partitioned_output)


Partitioned Output: [(0, [(np.float64(0.07457520048751265), np.float64(0.07457520048751265)), (np.float64(0.1684072247838545), np.float64(0.1684072247838545)), (np.float64(0.21547690728665703), np.float64(0.21547690728665703)), (np.float64(0.23642423983868743), np.float64(0.23642423983868743)), (np.float64(0.23772022031997508), np.float64(0.23772022031997508)), (np.float64(0.24608489711452874), np.float64(0.24608489711452874)), (np.float64(0.29029160991882286), np.float64(0.29029160991882286)), (np.float64(0.3000751345653565), np.float64(0.3000751345653565)), (np.float64(0.33940805504525096), np.float64(0.33940805504525096)), (np.float64(0.4146635592989292), np.float64(0.4146635592989292)), (np.float64(0.4447449853872948), np.float64(0.4447449853872948)), (np.float64(0.44983898084183904), np.float64(0.44983898084183904)), (np.float64(0.4899961436221161), np.float64(0.4899961436221161))]), (1, [(np.float64(0.5086300571891559), np.float64(0.5086300571891559)), (np.float64(0.5220998785740

# Упражнения
Упражнения взяты из Rajaraman A., Ullman J. D. Mining of massive datasets. – Cambridge University Press, 2011.

Для выполнения заданий переопределите функции RECORDREADER, MAP, REDUCE. 

Для модели распределённой системы может потребоваться переопределение функций PARTITION и COMBINER.

### Максимальное значение ряда

Разработайте MapReduce алгоритм, который находит максимальное число входного списка чисел.

In [26]:
from typing import Iterator
import numpy as np

d1 = np.random.rand(30)

def RECORDREADER():
    for value in d1:
        yield (value, None)

def MAP(value: float, _):
    yield (value, None)

def REDUCE(key: float, values: Iterator[None]):
    yield key

def flatten(nested_iterable):
    for iterable in nested_iterable:
        for element in iterable:
            yield element

def groupbykey(iterable):
    t = {}
    for (k2, v2) in iterable:
        t[k2] = t.get(k2, []) + [v2]
    return t.items()

def MapReduce(RECORDREADER, MAP, REDUCE):
    return flatten(map(lambda x: REDUCE(*x), groupbykey(flatten(map(lambda x: MAP(*x), RECORDREADER())))))

output = MapReduce(RECORDREADER, MAP, REDUCE)
max_value = max(output)
print(f"Максимальное значение: {max_value}")
print(d1)

Максимальное значение: 0.9866881591941783
[0.10441231 0.30589425 0.52364608 0.38852965 0.97687711 0.91584597
 0.69201001 0.35109474 0.22584654 0.6830471  0.40947403 0.5452787
 0.48745994 0.22286615 0.01792167 0.86870867 0.83879789 0.23507257
 0.32482869 0.26457599 0.32001808 0.85147695 0.04275044 0.83394052
 0.18517443 0.09596378 0.11159597 0.98668816 0.32371741 0.55871168]


### Арифметическое среднее

Разработайте MapReduce алгоритм, который находит арифметическое среднее.

$$\overline{X} = \frac{1}{n}\sum_{i=0}^{n} x_i$$


In [27]:
def REDUCE(key: float, values: Iterator[None]):
    total_sum = 0
    count = 0
    for v in values:
        total_sum += key  # суммируем все числа
        count += 1  # увеличиваем количество
    yield (total_sum, count)
    
output = MapReduce(RECORDREADER, MAP, REDUCE)
total_sum, count = next(output)
average = total_sum / count
print(f"Арифметическое среднее: {average}")

Арифметическое среднее: 0.10441231107112858


### Drop duplicates (set construction, unique elements, distinct)

Реализуйте распределённую операцию исключения дубликатов

In [28]:
d1 = [1, 1, 2, 2, 3, 4, 5]

def REDUCE(key: float, values: Iterator[None]):
    yield (key)
    
output = MapReduce(RECORDREADER, MAP, REDUCE)
unique_values = list(output)
print(f"Уникальные значения: {unique_values}")

Уникальные значения: [1, 2, 3, 4, 5]


## Операторы реляционной алгебры
### Selection (Выборка)

**The Map Function**: Для  каждого кортежа $t \in R$ вычисляется истинность предиката $C$. В случае истины создаётся пара ключ-значение $(t, t)$. В паре ключ и значение одинаковы, равны $t$.

**The Reduce Function:** Роль функции Reduce выполняет функция идентичности, которая возвращает то же значение, что получила на вход.



In [29]:
d1 = np.array([0.23, 0.76, 0.89, 0.12, 0.45, 0.67, 0.34, 0.56, 0.98, 0.34,
              0.23, 0.67, 0.78, 0.45, 0.56, 0.89, 0.76, 0.91, 0.43, 0.89,
              0.25, 0.64, 0.51, 0.73, 0.82, 0.39, 0.57, 0.61, 0.95, 0.33])

def predicate(value: float):
    return value > 0.5  # Предикат: выбираем только те элементы, которые больше 0.5

def RECORDREADER():
    for value in d1:
        if predicate(value):  # Применяем предикат перед маппингом
            yield (value, None)

def REDUCE(key: float, values: Iterator[None]):
    yield key

output = MapReduce(RECORDREADER, MAP, REDUCE)
selected_values = list(output)
print(f"Выбранные значения: {selected_values}")

Выбранные значения: [np.float64(0.76), np.float64(0.89), np.float64(0.67), np.float64(0.56), np.float64(0.98), np.float64(0.78), np.float64(0.91), np.float64(0.64), np.float64(0.51), np.float64(0.73), np.float64(0.82), np.float64(0.57), np.float64(0.61), np.float64(0.95)]


### Projection (Проекция)

Проекция на множество атрибутов $S$.

**The Map Function:** Для каждого кортежа $t \in R$ создайте кортеж $t′$, исключая  из $t$ те значения, атрибуты которых не принадлежат  $S$. Верните пару $(t′, t′)$.

**The Reduce Function:** Для каждого ключа $t′$, созданного любой Map задачей, вы получаете одну или несколько пар $(t′, t′)$. Reduce функция преобразует $(t′, [t′, t′, . . . , t′])$ в $(t′, t′)$, так, что для ключа $t′$ возвращается одна пара  $(t′, t′)$.

In [30]:
from typing import Iterator, Tuple

d1 = [
    (1, "Alice", 25, "Engineer"),
    (2, "Bob", 30, "Doctor"),
    (3, "Charlie", 22, "Artist"),
    (4, "David", 35, "Lawyer"),
    (5, "Eve", 28, "Scientist"),
    (6, "Frank", 40, "Engineer"),
]

# Атрибуты, которые оставляем (например, только имя и возраст)
S = [1, 2]  # Индексы атрибутов в кортежах

def RECORDREADER():
    for record in d1:
        yield tuple(record[i] for i in S), None  # Оставляем нужные атрибуты

# Функция REDUCE: убирает дубликаты
def REDUCE(key: Tuple, values: Iterator[None]):
    yield key  # Возвращаем только один экземпляр t'

output = MapReduce(RECORDREADER, MAP, REDUCE)
print(f"Проекция на атрибуты {S}: {list(output)}")

Проекция на атрибуты [1, 2]: [('Alice', 25), ('Bob', 30), ('Charlie', 22), ('David', 35), ('Eve', 28), ('Frank', 40)]


### Union (Объединение)

**The Map Function:** Превратите каждый входной кортеж $t$ в пару ключ-значение $(t, t)$.

**The Reduce Function:** С каждым ключом $t$ будет ассоциировано одно или два значений. В обоих случаях создайте $(t, t)$ в качестве выходного значения.

In [31]:
R = [(1, "Alice"), (2, "Bob"), (3, "Charlie")]
S = [(3, "Charlie"), (4, "David"), (5, "Eve")]

# Объединённый источник данных
def RECORDREADER():
    for record in R + S:  # Объединяем R и S
        yield (record, None)

# Функция REDUCE: просто возвращает ключ, так как он уже является кортежем (t, t)
def REDUCE(key: Tuple, values: Iterator[Tuple]):
    yield key

output = MapReduce(RECORDREADER, MAP, REDUCE)
union_result = list(output)
print(f"Объединение множеств R и S: {union_result}")


Объединение множеств R и S: [(1, 'Alice'), (2, 'Bob'), (3, 'Charlie'), (4, 'David'), (5, 'Eve')]


### Intersection (Пересечение)

**The Map Function:** Превратите каждый кортеж $t$ в пары ключ-значение $(t, t)$.

**The Reduce Function:** Если для ключа $t$ есть список из двух элементов $[t, t]$ $-$ создайте пару $(t, t)$. Иначе, ничего не создавайте.

In [32]:
# Функция REDUCE: оставляет только те ключи, которые встречаются дважды
def REDUCE(key: Tuple, values: Iterator[Tuple]):
    values_list = list(values)  # Преобразуем в список
    if len(values_list) > 1:  # Если есть два вхождения (из R и S)
        yield key  # Это пересечение

output = MapReduce(RECORDREADER, MAP, REDUCE)
intersection_result = list(output)
print(f"Пересечение множеств R и S: {intersection_result}")

Пересечение множеств R и S: [(3, 'Charlie')]


### Difference (Разница)

**The Map Function:** Для кортежа $t \in R$, создайте пару $(t, R)$, и для кортежа $t \in S$, создайте пару $(t, S)$. Задумка заключается в том, чтобы значение пары было именем отношения $R$ or $S$, которому принадлежит кортеж (а лучше, единичный бит, по которому можно два отношения различить $R$ or $S$), а не весь набор атрибутов отношения.

**The Reduce Function:** Для каждого ключа $t$, если соответствующее значение является списком $[R]$, создайте пару $(t, t)$. В иных случаях не предпринимайте действий.

In [33]:
def RECORDREADER():
    for number in R:
        yield (number, "R") 
    for number in S:
        yield (number, "S") 
        
def MAP(value, key):
    yield (value, key)

def REDUCE(key, values):
    if values == ["R"]:
        yield (key, key)

output = MapReduce(RECORDREADER, MAP, REDUCE)
difference_result = list(output)
print(f"Разность множеств R - S: {difference_result}")

Разность множеств R - S: [((1, 'Alice'), (1, 'Alice')), ((2, 'Bob'), (2, 'Bob'))]


### Natural Join

**The Map Function:** Для каждого кортежа $(a, b)$ отношения $R$, создайте пару $(b,(R, a))$. Для каждого кортежа $(b, c)$ отношения $S$, создайте пару $(b,(S, c))$.

**The Reduce Function:** Каждый ключ $b$ будет асоциирован со списком пар, которые принимают форму либо $(R, a)$, либо $(S, c)$. Создайте все пары, одни, состоящие из  первого компонента $R$, а другие, из первого компонента $S$, то есть $(R, a)$ и $(S, c)$. На выходе вы получаете последовательность пар ключ-значение из списков ключей и значений. Ключ не нужен. Каждое значение, это тройка $(a, b, c)$ такая, что $(R, a)$ и $(S, c)$ это принадлежат входному списку значений.

In [34]:
R = [(1, "Alice"), (2, "Bob"), (3, "Charlie")]
S = [("Charlie", 10), ("David", 20), ("Bob", 30)]

def RECORDREADER():
    for (a, b) in R:
        yield (b, None)
    for (b, c) in S:
        yield (b, None)

def MAP(value: float, _):
    yield (value, None)

def REDUCE(key, values):
    r_vals = [a for (a, b) in R if b == key]
    s_vals = [c for (b, c) in S if b == key]
    
    for a in r_vals:
        for c in s_vals:
            yield (a, key, c)


output = MapReduce(RECORDREADER, MAP, REDUCE)
natural_join_result = list(output)
print("Результат Natural Join:", natural_join_result)

Результат Natural Join: [(2, 'Bob', 30), (3, 'Charlie', 10)]


### Grouping and Aggregation (Группировка и аггрегация)

**The Map Function:** Для каждого кортежа $(a, b, c$) создайте пару $(a, b)$.

**The Reduce Function:** Ключ представляет ту или иную группу. Примение аггрегирующую операцию $\theta$ к списку значений $[b1, b2, . . . , bn]$ ассоциированных с ключом $a$. Возвращайте в выходной поток $(a, x)$, где $x$ результат применения  $\theta$ к списку. Например, если $\theta$ это $SUM$, тогда $x = b1 + b2 + · · · + bn$, а если $\theta$ is $MAX$, тогда $x$ это максимальное из значений $b1, b2, . . . , bn$.

In [35]:
data = [(1, 10, "A"), (1, 20, "B"), (2, 30, "C"), (2, 40, "D")]

def MAP(key,value):
    yield (key, value)

def RECORDREADER():
    for item in data:
        yield (item[0], item[1])

def REDUCE(key: float, values: Iterator[None], operation="SUM"):
    if operation == "SUM":
        yield (key, sum(values)) 
    if operation == "MAX":
        yield (key, max(values)) 

output_sum = MapReduce(RECORDREADER, MAP, REDUCE) 
aggregation_result_sum = list(output_sum)
print("Результат агрегации (SUM):", aggregation_result_sum)

output_max = MapReduce(RECORDREADER, MAP, lambda key, values: REDUCE(key, values, operation="MAX"))
aggregation_result_max = list(output_max)
print("Результат агрегации (MAX):", aggregation_result_max)

Результат агрегации (SUM): [(1, 30), (2, 70)]
Результат агрегации (MAX): [(1, 20), (2, 40)]


## Вычисление TF-IDF (Term Frequency – Inverse Document Fraquency)

Реализуется в три этапа:

**Этап 1:** Частота слова в документе

**Этап 2:** Количество документов, в которых встречается слово

**Этап 3:** Расчёт TF-IDF

In [50]:
annotations = [
    """
    Streaming data is the data from sensors as well as other real-time surveillance systems. Distributed stream processing systems are the software that manages such data. Such frameworks have to deliver outcomes on the go instantly. They are susceptible to delay and malfunction or system failures. The system must be tolerant of faults and always accessible. Many variables, such as improved network arrival rates, node failures, and so on, disrupt the system's reliability. Some operators need to be relocated online from one physical resource to another to manage or reimburse a slow or failing node. In this study, we propose a co-location based systematic migration heuristic for live operator migration between physical resources using a migration map revised with costs for each migration. The suggested method evaluates continuous operator performance patterns and makes online scheduling decisions based on the same. The decisions include migrating operators during a node failure or straggling.
    """,
    """
    Distributed stream processing engines are designed with a focus on scalability to process big data volumes in a continuous manner. We present the Theodolite method for benchmarking the scalability of distributed stream processing engines. Core of this method is the definition of use cases that microservices implementing stream processing have to fulfill. For each use case, our method identifies relevant workload dimensions that might affect the scalability of a use case. We propose to design one benchmark per use case and relevant workload dimension. We present a general benchmarking framework, which can be applied to execute the individual benchmarks for a given use case and workload dimension. Our framework executes an implementation of the use case's dataflow architecture for different workloads of the given dimension and various numbers of processing instances. This way, it identifies how resources demand evolves with increasing workloads. Within the scope of this paper, we present 4 identified use cases, derived from processing Industrial Internet of Things data, and 7 corresponding workload dimensions. We provide implementations of 4 benchmarks with Kafka Streams and Apache Flink as well as an implementation of our benchmarking framework to execute scalability benchmarks in cloud environments. We use both for evaluating the Theodolite method and for benchmarking Kafka Streams' and Flink's scalability for different deployment options.
    """,
    """
    Batch and stream processing are separately and efficiently applied in many applications. However, some newer data-driven applications such as the Internet of Things and cloud computing call for hybrid processing approaches in order to handle the speed and accuracy required for processing such complex data. In this paper, we propose a Hybrid Distributed Batch-Stream (HDBS) architecture for anomaly detection in real-time data. The hybrid architecture, while benefiting from the accuracy provided by batch processing, also enjoys the speed and real-time features of stream processing. In the proposed architecture, our focus is on the algorithmic aspects of hybrid processing including the interaction models between batch and stream processing units, the characteristics of batch and stream machine learning algorithms and the principles of merging the results of different processing units. The driving idea of such combination is that the results of batch and stream processing units are complementary with each other, as one of them constructs accurate models based on previous data, and the other one is capable of processing new stream data in real-time. Furthermore, we propose a generalized version of the HDBS with respect to its algorithms and communication policy levels. In the generalized HDBS architecture, we address the various aspects of the interaction between the batch and stream processing units, and the merging operations to produce the final results. the evaluations of the proposed architecture using various criteria (accuracy, space complexity, and time complexity) demonstrate that the accuracy of the proposed method is higher than the accuracy of the batch processing methods, its time complexity is also similar to one of the stream processing methods and much less than the batch processing methods, which makes our proposed architecture an efficient and practical solution for real-time anomaly detection.
    """,
    """
    There have been increasing demands for real time processing of the ever-growing data. In order to meet this requirement and ensure the reliable processing of streaming data, a variety of distributed stream processing architectures and platforms have been developed, which handles the fundamental task of allocating processing tasks to the currently available physical resources and routing streaming data between these resources. However, many stream processing systems lack an intelligent scheduling mechanism, in which their default schedulers allocate tasks without taking resource demands and availability, or the transfer latency between resources into consideration. Besides, stream processing has a strict request for latency. Thus it is important to give latency guarantee for distributed stream processing. In this paper, we propose two new algorithms for stream processing with latency guarantee, both the algorithms consider transfer latency and resource demand in task allocation. Both algorithms can guarantee latency constraints. Algorithm AHA reduces more than 21.3% and 58.9% resources compared with the greedy and the round-robin algorithms, and algorithm PHA further improves the resource utilization to 32.1% and 73.2%.
    """,
    """
    In the era of Big Data, typical architecture of distributed real-time stream processing systems is the combination of Flume, Kafka, and Storm. As a kind of distributed message system, Kafka has the characteristics of horizontal scalability and high throughput, which is manly deployed in many areas in order to address the problem of speed mismatch between message producers and consumers. When using Kafka, we need to quickly receive data sent by producers. In addition, we need to send data to consumers quickly. Therefore, the performance of Kafka is of critical importance to the performance of the whole stream processing system. In this paper, we propose the improved design of real-time stream processing systems, and focus on improving the Kafka’s data loading process. We use Kafka cat to transfer data from the source to Kafka topic directly, which can reduce the network transmission. We also utilize the memory file system to accelerate the process of data loading, which can address the bottleneck and performance problems caused by disk I/O. Extensive experiments are conducted to evaluate the performance, which show the superiority of our improved design.
    """,
    """
    In this paper, nearly 40 commonly used deep neural network(DNN) models are selected, and their cross-platform and cross-inference frameworks are deeply analysed. The main metrics of accuracy, the total number of model parameters, the computational complexity, the accuracy density, the inference time, the memory consumption and other related parameters are used to measure their performance. The heterogeneous computing experiment is implemented on both the Google Colab cloud computing platform and the Jetson Nano embedded edge computing platform. The obtained performance is compared with that of two previous computing platforms: a workstation equipped with an NVIDIA Titan X Pascal and an embedded system based on an NVIDIA Jetson TX1 board. In addition, on the Jetson Nano embedded edge computing platform, different inference frameworks are investigated to evaluate the inference efficiency of the DNN models. Regression models are established to characterize the variation in the computing performance of different DNN classification algorithms so that the inference results of unknown models can be estimated. ANOVA methods are proposed to quantify the differences between models. The experimental results have important guiding significance for the better selection, deployment and application of DNN models in practice. Codes are available at this https URL https://github.com/Foreverzfy/Model-Test.
    """,
    """
    Unmanned Aerial Vehicles (UAVs), which can operate autonomously in dynamic and complex environments, are becoming increasingly common. Deep learning techniques for motion control have recently taken a major qualitative step since vision-based inference tasks can be executed directly on edge. The goal is to fully integrate the machine learning (ML) element into small UAVs. However, given the limited payload capacity and energy available on small UAVs, integrating computing resources sufficient to host ML and vehicle control functions is still challenging. This paper presents a modular and generic system that can control the UAV by evaluating vision-based ML tasks directly inside the resource-constrained UAV. Two different vision-based navigation configurations were tested and demonstrated. The first configuration implements an autonomous landing site detection system, tested with two models based on LeNet-5 and MobileNetV2, respectively. This allows the UAV to change its planned path accordingly and approach the target to land. Moreover, a model for people detection based on a custom MobileNetV2 network was evaluated in the second configuration. Finally, the execution time and power consumption were measured and compared with a cloud computing approach. The results show the ability of the developed system to dynamically react to the environment to provide the necessary maneuver after detecting the target exploiting only the constrained computational resources of the UAV controller. Furthermore, we demonstrated that moving to the edge, instead of using cloud computing inference, decreases the energy requirement of the system without reducing the quality of service.
    """,
    """
    With the continuous development of Internet of Things (IoT) and the overwhelming explosion of Big Data, edge computing serves as an efficient computing mode for time stringent data processing, which can bypass the constraints of network bandwidth and delay, and has been one of the foundation of interconnected applications. Although edge computing has gradually become one of bridges between cloud computing centers and mobile terminals, the literature still lacks a thorough review on the recent advances in edge computing platforms. In this paper, we firstly introduce the definition of edge computing and advantages of edge computing platform. And then, we summarize the key technologies of constructing an edge computing platform, and propose a general framework for edge computing platform. The role of distributed storage management systems in building edge computing platform is elaborated in detail. Furthermore, we give some applications to illustrate how to use third-party edge computing platforms to build specific applications. Finally, we briefly outline current open issues of edge computing platform based on our literature survey.
    """
]

In [51]:
from typing import Iterator
import numpy as np
import re
import math
from collections import defaultdict
from tabulate import tabulate

In [52]:
def clean_text(text: str):
    return re.sub(r'[^\w\s]', '', text.lower())

cleaned_docs = [clean_text(doc) for doc in annotations]
N = len(cleaned_docs)

In [59]:
# TF
def tf_record_reader():
    for doc_id, text in enumerate(cleaned_docs):
        yield (doc_id, text)

def tf_map(doc_id: int, text: str):
    words = text.split()
    total = len(words)
    for word in words:
        yield ((doc_id, word), 1/total)

def tf_reducer(key, values):
    yield (key, sum(values))

tf_results = list(MapReduce(tf_record_reader, tf_map, tf_reducer))
tf_dict = {(doc, word): val for (doc, word), val in tf_results}

# IDF 
def idf_record_reader():
    for doc_id, text in enumerate(cleaned_docs):
        yield (doc_id, text)

def idf_map(doc_id: int, text: str):
    for word in set(text.split()):
        yield (word, doc_id)

def idf_reducer(word: str, doc_ids: Iterator[int]):
    yield (word, math.log(N / len(set(doc_ids))))

idf_results = dict(MapReduce(idf_record_reader, idf_map, idf_reducer))

In [60]:
def print_document_stats(doc_id):
    print(f"\n{'='*40}\nДокумент {doc_id+1}\n{'='*40}")
    
#TF
    tf_items = [(word, val) for (d, word), val in tf_dict.items() if d == doc_id]
    tf_top = sorted(tf_items, key=lambda x: (-x[1], x[0]))[:5]
    print("\nTerm Frequency (TF):")
    print(tabulate(tf_top, headers=["Слово", "TF"], tablefmt="grid", floatfmt=".4f"))
    
#IDF
    doc_words = {word for (d, word) in tf_dict.keys() if d == doc_id}
    idf_items = [(word, idf_results[word]) for word in doc_words if word in idf_results]
    idf_top = sorted(idf_items, key=lambda x: (-x[1], x[0]))[:5]
    print("\nInverse Document Frequency (IDF):")
    print(tabulate(idf_top, headers=["Слово", "IDF"], tablefmt="grid", floatfmt=".4f"))
    
#TF-IDF
    tfidf_items = [(word, score) for word, score in tfidf[doc_id].items()]
    tfidf_top = sorted(tfidf_items, key=lambda x: (-x[1], x[0]))[:5]
    print("\nTF-IDF:")
    print(tabulate(tfidf_top, headers=["Слово", "TF-IDF"], tablefmt="grid", floatfmt=".4f"))

#Вывод статистики для каждого документа
for doc_id in range(N):
    print_document_stats(doc_id)

#Глобальный TF-IDF
print("\n"+ "="*40 + "\nГлобальный топ TF-IDF\n" + "="*40)
all_tfidf = [(word, score) for doc in tfidf.values() for word, score in doc.items()]
global_tfidf = sorted(all_tfidf, key=lambda x: (-x[1], x[0]))[:5]
print(tabulate(global_tfidf, headers=["Слово", "TF-IDF"], tablefmt="grid", floatfmt=".4f"))


Документ 1

Term Frequency (TF):
+-----------+--------+
| Слово     |     TF |
+===========+========+
| the       | 0.0533 |
+-----------+--------+
| to        | 0.0333 |
+-----------+--------+
| a         | 0.0267 |
+-----------+--------+
| and       | 0.0267 |
+-----------+--------+
| migration | 0.0267 |
+-----------+--------+

Inverse Document Frequency (IDF):
+------------+--------+
| Слово      |    IDF |
+============+========+
| accessible | 2.0794 |
+------------+--------+
| always     | 2.0794 |
+------------+--------+
| another    | 2.0794 |
+------------+--------+
| arrival    | 2.0794 |
+------------+--------+
| colocation | 2.0794 |
+------------+--------+

TF-IDF:
+-----------+----------+
| Слово     |   TF-IDF |
+===========+==========+
| migration |   0.0555 |
+-----------+----------+
| node      |   0.0416 |
+-----------+----------+
| or        |   0.0370 |
+-----------+----------+
| decisions |   0.0277 |
+-----------+----------+
| failures  |   0.0277 |
+----------